In [155]:
# Import required librairies
import os
from os import listdir, stat, makedirs, mkdir, walk, remove, pardir, rename
from os.path import isdir, isfile, join, splitext, getmtime, basename, normpath, exists, expanduser, split, dirname, getsize, abspath
from blackfynn import Blackfynn
import json 
from datetime import datetime, timezone
import pandas as pd
import pathlib
import shutil
import gevent
import urllib.request
import copy
import re
import platform
import subprocess

# Define constants
userpath = expanduser("~")

In [156]:
def bf_get_current_user_permission(bf, myds):

    """
    Function to get the permission of currently logged in user for a selected dataset

    Args:
        bf: logged Blackfynn acccount (dict)
        myds: selected Blackfynn dataset (dict)
    Output:
        permission of current user (string)
    """

    try:
        selected_dataset_id = myds.id
        user_role = bf._api._get('/datasets/' + str(selected_dataset_id) + '/role')['role']

        return user_role

    except Exception as e:
        raise e

In [157]:
def dump(obj):
   for attr in dir(obj):
       if hasattr( obj, attr ):
           print( "obj.%s = %s" % (attr, getattr(obj, attr)))

In [158]:
high_level_sparc_folders = ["code", "derivative", "docs", "primary", "protocol", "source"]
manifest_sparc = ["manifest.xlsx", "manifest.csv"]
high_level_metadata_sparc = ['submission.xlsx', 'submission.csv', 'submission.json',
  'dataset_description.xlsx', 'dataset_description.csv', 'dataset_description.json',
  'subjects.xlsx', 'subjects.csv', 'subjects.json',
  'samples.xlsx', 'samples.csv', 'samples.json',
  'README.txt', 'CHANGES.txt']

def bf_import_dataset_to_json(soda_json_structure, requested_sparc_only = True):
    """
    Function for importing blackfynn data files info into the "dataset-structure" key of the soda json structure, 
    including metadata from any existing manifest files in the high-level folders 
    (name, id, timestamp, description, additional metadata)

    Args:
        soda_json_structure: soda structure with bf account and dataset info available
    Output:
        same soda structure with blackfyn data file info included under the "dataset-structure" key
    """
    def verify_file_name(item_name, file_name):
        filename, file_extension = os.path.splitext(file_name)
        if file_extension in bf_recognized_file_extensions:
            return item_name + file_extension
        else:
            return file_name
        
    def recursive_dataset_import(my_item, dataset_folder, metadata_files, my_folder_name, my_level, manifest_dict):
        col_count = 0
        file_count = 0

        for item in my_item:
            print(item)
            if item.type == "Collection":
                #dump(item)
                if "folders" not in dataset_folder:
                    dataset_folder["folders"] = {}
                if "files" not in dataset_folder:
                    dataset_folder["files"] = {}
                col_count += 1
                folder_name = item.name
                if my_level == 0 and folder_name not in high_level_sparc_folders and requested_sparc_only: #only import SPARC folders
                    continue
                if col_count == 1:
                    #dataset_folder["folders"] = {}
                    level  = my_level + 1
                dataset_folder["folders"][folder_name] = {"type": "bf", "action": ["existing"], "path": item.id}
                sub_folder = dataset_folder["folders"][folder_name]
                if "folders" not in sub_folder:
                    sub_folder["folders"] = {}
                if "files" not in sub_folder:
                    sub_folder["files"] = {}
                recursive_dataset_import(item, sub_folder, metadata_files, folder_name, level, manifest_dict)
            else:
                if "folders" not in dataset_folder:
                    dataset_folder["folders"] = {}
                if "files" not in dataset_folder:
                    dataset_folder["files"] = {}
                package_id = item.id
                file_details = bf._api._get('/packages/' + str(package_id) + '/view')
                #print(item)
                #print(file_details);
                file_name = file_details[0]["content"]["name"]
                
                if my_level == 0 and file_name in high_level_metadata_sparc:
                    metadata_files[file_name] = {"type": "bf", "action": ["existing"], "path": item.id} 
                    
                else:
                    file_count += 1
                    #if file_count == 1:
                        #dataset_folder["files"] = {}
#                     if my_level == 0:
#                         dataset_folder["files"][file_name] = {"type": "bf", "action": ["existing"], "path": item.id}  
                    if my_level == 1 and file_name in manifest_sparc:
                        file_id = file_details[0]["content"]["id"]
                        manifest_url = bf._api._get('/packages/' + str(package_id) + '/files/' + str(file_id))
                        df = pd.read_excel(manifest_url['url'])
                        manifest_dict[my_folder_name] = df
                    else:
                        timestamp = file_details[0]["content"]["updatedAt"]
                        dataset_folder["files"][file_name] = {"type": "bf", "action": ["existing"], "path": item.id, "timestamp": timestamp}

    def recursive_manifest_info_import(my_folder, my_relative_path, manifest_df):
        
        if "files" in my_folder.keys():
            for file_key, file in my_folder["files"].items():
                    filename = join(my_relative_path, file_key)
                    colum_headers = manifest_df.columns.tolist()
                    if filename in list(manifest_df["filename"].values):
                        if "description" in colum_headers:
                            mydescription = manifest_df[manifest_df['filename'] == filename]["description"].values[0]
                            if mydescription:
                                file["description"] = mydescription
                        if "Additional Metadata" in colum_headers:
                            my_additional_medata = manifest_df[manifest_df['filename'] == filename]["Additional Metadata"].values[0]
                            if mydescription:
                                file["additional-metadata"] = my_additional_medata

        if "folders" in my_folder.keys():
            for folder_key, folder in my_folder["folders"].items():
                relative_path = join(my_relative_path, folder_key)
                recursive_manifest_info_import(folder, relative_path, manifest_df)
    
    # START
    
    error = []
    
    # check that the blackfynn account is valid
    try:
        bf_account_name = soda_json_structure["bf-account-selected"]["account-name"]
    except Exception as e:
        raise e
        
    try:
        bf = Blackfynn(bf_account_name)
    except Exception as e:
        error.append('Error: Please select a valid Blackfynn account')
        raise Exception(error)  

    # check that the blackfynn dataset is valid
    try:
        bf_dataset_name = soda_json_structure["bf-dataset-selected"]["dataset-name"]
    except Exception as e:
        raise e
    try:
        myds = bf.get_dataset(bf_dataset_name)
    except Exception as e:
        error.append('Error: Please select a valid Blackfynn dataset')
        raise Exception(error)
    
    # check that the user has permission to edit this dataset
    try:
        role = bf_get_current_user_permission(bf, myds)
        if role not in ['owner', 'manager', 'editor']:
            curatestatus = 'Done'
            error.append("Error: You don't have permissions for uploading to this Blackfynn dataset")
            raise Exception(error)
    except Exception as e:
        raise e
    
    try:
        # import files and folders in the soda json structure
        soda_json_structure["dataset-structure"] = {}
        soda_json_structure["metadata-files"] = {}
        dataset_folder = soda_json_structure["dataset-structure"]
        metadata_files = soda_json_structure["metadata-files"]
        manifest_dict = {}
        level = 0
        folder_name = ""
        #dump(myds)
        recursive_dataset_import(myds, dataset_folder, metadata_files, folder_name, level, manifest_dict)
        
        #remove metadata files keys if empty
        metadata_files = soda_json_structure["metadata-files"]
        if not metadata_files:
            del soda_json_structure['metadata-files']
        
        # pull information from the manifest files if they satisfy the SPARC format
        if "folders" in dataset_folder.keys():
            for folder_key in manifest_dict.keys():
                manifest_df = manifest_dict[folder_key]
                manifest_df = manifest_df.fillna('')  
                colum_headers = manifest_df.columns.tolist()
                folder = dataset_folder["folders"][folder_key]
                if "filename" in colum_headers:
                    if "description" in colum_headers or "Additional Metadata" in colum_headers:
                        relative_path = ""
                        recursive_manifest_info_import(folder, relative_path, manifest_df)

        success_message = "Data files under a valid high-level SPARC folders have been imported"
        return [soda_json_structure, success_message]
    
    except Exception as e:
        raise e

In [159]:
soda_json_structure = {
    "bf-account-selected": {
        "account-name": "calmilinux"
    },
    
    "bf-dataset-selected": {
        "dataset-name": "testddataset"
    },
    
    "dataset-structure": {},
    
    "metadata-files": {},
    
    "manifest-files":{},
    
    "generate-dataset": {},
}

In [160]:
bf_import_dataset_to_json(soda_json_structure)

<Collection name='primary' id='N:collection:95155ae6-df69-45f4-8493-0b988720c92a'>
<DataPackage name='manifest' id='N:package:f0573ab9-e78d-407c-b6d4-085f40cf5d7f'>
<DataPackage name='submission' id='N:package:3b504e70-58ca-4bfb-8c80-f214f32ccaa0'>
<DataPackage name='subjects' id='N:package:41295a11-4fe0-4d4d-95fb-26b50646f74b'>
<DataPackage name='dataset_description' id='N:package:69a01be0-42c3-4129-9898-4aa47734b4ea'>
<Collection name='source' id='N:collection:2994d497-37c0-41c9-b88a-1daff9a7c0bc'>
<Collection name='png' id='N:collection:252325e6-ee96-4a79-9379-1b3536d33877'>
<DataPackage name='soda_icon (1)' id='N:package:ed1ae9a2-5067-428a-9dde-65a76b5f2732'>
<DataPackage name='soda_icon2' id='N:package:ab3029d9-31f0-49d5-9e37-97cd1066692a'>
<DataPackage name='icon.ico' id='N:package:1d277d08-b129-4684-b611-1d33ddcdda2a'>
<Collection name='code' id='N:collection:cdeabbcd-47b6-4f0b-b500-56f5e67fc0b6'>
<Collection name='fhj' id='N:collection:67f32258-459e-4db6-a884-859ab38b2efb'>
<Da

[{'bf-account-selected': {'account-name': 'calmilinux'},
  'bf-dataset-selected': {'dataset-name': 'testddataset'},
  'dataset-structure': {'folders': {'primary': {'type': 'bf',
     'action': ['existing'],
     'path': 'N:collection:95155ae6-df69-45f4-8493-0b988720c92a',
     'folders': {},
     'files': {}},
    'source': {'type': 'bf',
     'action': ['existing'],
     'path': 'N:collection:2994d497-37c0-41c9-b88a-1daff9a7c0bc',
     'folders': {'png': {'type': 'bf',
       'action': ['existing'],
       'path': 'N:collection:252325e6-ee96-4a79-9379-1b3536d33877',
       'folders': {},
       'files': {'soda_icon.png': {'type': 'bf',
         'action': ['existing'],
         'path': 'N:package:ed1ae9a2-5067-428a-9dde-65a76b5f2732',
         'timestamp': '2020-12-21T19:16:17.28051Z'},
        'soda_icon2.png': {'type': 'bf',
         'action': ['existing'],
         'path': 'N:package:ab3029d9-31f0-49d5-9e37-97cd1066692a',
         'timestamp': '2020-12-21T19:16:18.86915Z'},
        

In [91]:
bf = Blackfynn("calmilinux")
file = bf.get("N:package:7e2651f9-3124-4ade-aff1-9d36bf3c45e2")
print(file)
package_id = file.id
file_details = bf._api._get('/packages/' + str(package_id) + '/view')
print(file_details);
file_name = file_details[0]["content"]["name"]

<DataPackage name='test.odb' id='N:package:7e2651f9-3124-4ade-aff1-9d36bf3c45e2'>
[{'content': {'packageId': 'N:package:7e2651f9-3124-4ade-aff1-9d36bf3c45e2', 'name': 'test.odb', 'fileType': 'GenericData', 'filename': 'test.odb', 's3bucket': 'prod-storage-blackfynn', 's3key': '1079/99f7dbca-9bd8-4417-add4-1f19b3b3804a/test.odb', 'objectType': 'source', 'size': 24, 'createdAt': '2020-12-21T23:28:11.682467Z', 'updatedAt': '2020-12-21T23:28:11.934797Z', 'id': 75120, 'checksum': {'chunkSize': 5242880, 'checksum': '5d9ae25c5ecad15711410c191e4e8630ffe34c2cdba5e901738f4ee270977999'}}}]


In [96]:
soda_json_structure["metadata-files"]

{'submission.csv': {'type': 'bf',
  'path': 'N:package:3b504e70-58ca-4bfb-8c80-f214f32ccaa0'}}

In [88]:
file.name ="testname";
file.update();

In [142]:
bf = Blackfynn("calmilinux")
file = bf.get("N:collection:cdeabbcd-47b6-4f0b-b500-56f5e67fc0b6")
print(file)
file.name = "code"
file.update()
print(file)
temp = file.create_collection("fhj")
print(temp)

<Collection name='code' id='N:collection:cdeabbcd-47b6-4f0b-b500-56f5e67fc0b6'>
<Collection name='code' id='N:collection:cdeabbcd-47b6-4f0b-b500-56f5e67fc0b6'>
<Collection name='fhj' id='N:collection:67f32258-459e-4db6-a884-859ab38b2efb'>


In [134]:
def recursive_file_delete(folder):
    for item in list(folder["files"]):
        #print(item)
        if "deleted" in folder["files"][item]['action']:
            print(item)
            del folder["files"][item]
    for item in list(folder["folders"]):
        recursive_file_delete(folder["folders"][item])
    return

In [135]:
sdbf = bf_import_dataset_to_json(soda_json_structure)[0]["dataset-structure"]

<Collection name='primary' id='N:collection:95155ae6-df69-45f4-8493-0b988720c92a'>
<DataPackage name='manifest' id='N:package:f0573ab9-e78d-407c-b6d4-085f40cf5d7f'>
<DataPackage name='submission' id='N:package:3b504e70-58ca-4bfb-8c80-f214f32ccaa0'>
<DataPackage name='subjects' id='N:package:41295a11-4fe0-4d4d-95fb-26b50646f74b'>
<DataPackage name='dataset_description' id='N:package:69a01be0-42c3-4129-9898-4aa47734b4ea'>
<Collection name='source' id='N:collection:2994d497-37c0-41c9-b88a-1daff9a7c0bc'>
<Collection name='png' id='N:collection:252325e6-ee96-4a79-9379-1b3536d33877'>
<DataPackage name='soda_icon (1)' id='N:package:ed1ae9a2-5067-428a-9dde-65a76b5f2732'>
<DataPackage name='soda_icon2' id='N:package:ab3029d9-31f0-49d5-9e37-97cd1066692a'>
<DataPackage name='icon.ico' id='N:package:1d277d08-b129-4684-b611-1d33ddcdda2a'>
<Collection name='code' id='N:collection:cdeabbcd-47b6-4f0b-b500-56f5e67fc0b6'>
<DataPackage name='test.odb' id='N:package:7e2651f9-3124-4ade-aff1-9d36bf3c45e2'>


In [162]:
sd = {"bf-account-selected":{"account-name":"calmilinux"},
      "bf-dataset-selected":{"dataset-name":"testddataset"},
      "dataset-structure":{"folders":{"code":{"type":"bf","action":["existing"],"path":"N:collection:cdeabbcd-47b6-4f0b-b500-56f5e67fc0b6","folders":{"fhj":{"type":"bf","action":["existing"],"path":"N:collection:67f32258-459e-4db6-a884-859ab38b2efb","folders":{},"files":{"icon.ico":{"path":"/home/dev/Desktop/SODA/src/assets/app-icon/png/icon.ico","type":"local","description":"","additional-metadata":"","action":["new"]}}}},"files":{"test.odb":{"type":"bf","action":["existing"],"path":"N:package:7e2651f9-3124-4ade-aff1-9d36bf3c45e2","timestamp":"2020-12-21T23:28:11.934797Z"},"soda_icon2.png":{"type":"bf","action":["existing"],"path":"N:package:ce7e3c3d-7111-4c4c-8a7c-ef1554dcaa9f","timestamp":"2020-12-21T18:41:52.399846Z"},"soda_icon.png":{"type":"bf","action":["existing"],"path":"N:package:5294f17d-0de1-4023-9322-4eae7c60eff0","timestamp":"2020-12-21T18:41:51.311691Z"},"testname2.jpg":{"type":"bf","action":["existing"],"path":"N:package:9265b437-a0f3-4768-bf33-fbe50d3291ec","timestamp":"2020-12-21T18:41:50.563632Z"}}},"primary":{"type":"bf","action":["existing"],"path":"N:collection:95155ae6-df69-45f4-8493-0b988720c92a","folders":{},"files":{"soda_icon.png":{"path":"/home/dev/Desktop/SODA/src/assets/app-icon/png/soda_icon.png","type":"local","description":"","additional-metadata":"","action":["new"]}}},"source":{"type":"bf","action":["existing"],"path":"N:collection:2994d497-37c0-41c9-b88a-1daff9a7c0bc","folders":{"png":{"type":"bf","action":["existing"],"path":"N:collection:252325e6-ee96-4a79-9379-1b3536d33877","folders":{},"files":{"soda_icon (1).png":{"type":"bf","action":["existing"],"path":"N:package:ed1ae9a2-5067-428a-9dde-65a76b5f2732","timestamp":"2020-12-21T19:16:17.28051Z"},"soda_icon2.png":{"type":"bf","action":["existing"],"path":"N:package:ab3029d9-31f0-49d5-9e37-97cd1066692a","timestamp":"2020-12-21T19:16:18.86915Z"},"icon.ico":{"type":"bf","action":["existing"],"path":"N:package:1d277d08-b129-4684-b611-1d33ddcdda2a","timestamp":"2020-12-21T19:16:18.124946Z"}}}},"files":{}}}},
      "metadata-files":{
          "submission.csv":{"type":"bf","action":["existing"],"path":"N:package:3b504e70-58ca-4bfb-8c80-f214f32ccaa0"},
          "subjects.csv":{"type":"bf","action":["existing"],"path":"N:package:41295a11-4fe0-4d4d-95fb-26b50646f74b"},
          "dataset_description.csv":{"type":"bf","action":["existing"],"path":"N:package:69a01be0-42c3-4129-9898-4aa47734b4ea"}},
      "metadata-files":{
          "submission.csv":{"type":"bf","action":["existing"],"path":"N:package:3b504e70-58ca-4bfb-8c80-f214f32ccaa0"},
          "subjects.csv":{"type":"bf","action":["existing"],"path":"N:package:41295a11-4fe0-4d4d-95fb-26b50646f74b"},
          "dataset_description.csv":{"type":"bf","action":["existing"],"path":"N:package:69a01be0-42c3-4129-9898-4aa47734b4ea"}},
      "manifest-files":{},
      "generate-dataset":{},
      "starting-point":"bf"}

In [163]:
sdmy = sd["dataset-structure"]

In [164]:
mysd = sdmy.copy()
bfsd = sdbf.copy()

In [165]:
# Delete any files on blackfynn that have been marked as deleted
def recursive_file_delete(folder):
    for item in list(folder["files"]) or []:
        if "deleted" in folder["files"][item]['action']:
            file = bf.get(folder["files"][item]['path'])
            file.delete()
            del folder["files"][item]

    for item in list(folder["folders"]):
        recursive_file_delete(folder["folders"][item])
    return

# Add a new key containing the path to all the files and folders on the 
# local data structure.
# Allows us to see if the folder path of a specfic file already 
# exists on blackfynn.
def recursive_item_path_create(folder, path):
    for item in list(folder["files"]):
        if "folderpath" not in folder["files"][item]:
            folder["files"][item]['folderpath'] = path[:]

    for item in list(folder["folders"]):
        if "folderpath" not in folder["folders"][item]:
            folder["folders"][item]['folderpath'] = path[:]
            folder["folders"][item]['folderpath'].append(item)
        recursive_item_path_create(folder["folders"][item], folder["folders"][item]['folderpath'][:])

    return

# Check and create any non existing folders for the file move process
def recursive_check_and_create_bf_file_path(folderpath, index, bfsd):
    folder = folderpath[index]

    if folder not in bfsd["folders"]:
        if (index == 0):
            new_folder = ds.create_collection(folder)
        else:
            current_folder = bf.get(bfsd["path"])
            new_folder = current_folder.create_collection(folder)
        bfsd["folders"][folder] = {"type": "bf", "action": ["existing"], "path": new_folder.id, "folders":{}, "files":{}}

    index += 1

    if index < len(folderpath):
        recursive_check_and_create_bf_file_path(folderpath, index, bfsd["folders"][folder])
    else:
        return bfsd["folders"][folder]["path"]

# Check for any files that have been moved and verify paths before moving
def recursive_check_moved_files(folder):
    for item in list(folder["files"]):
        if "moved" in folder["files"][item]['action'] and folder["files"][item]["type"] == "bf":
            new_folder_id = ""
            new_folder_id = recursive_check_and_create_bf_file_path(folder["files"][item]["folderpath"].copy(), 0, bfsd)
            destination_folder = bf.get(new_folder_id)
            bf.move(destination_folder, folder["files"][item]["path"])

    for item in list(folder["folders"]):
        recursive_check_moved_files(folder["folders"][item])

    return

# Rename any files that exist on blackfynn
def recursive_file_rename(folder):
    for item in list(folder["files"]):
        if "renamed" in folder["files"][item]['action'] and folder["files"][item]["type"] == "bf":
            file = bf.get(folder["files"][item]["path"])
            file.name = item
            file.update()

    for item in list(folder["folders"]):
        recursive_file_rename(folder["folders"][item])

    return

# Delete any stray folders that exist on blackfynn
# Only top level files are deleted since the api deletes any 
# files and folders that exist inside.
def recursive_folder_delete(folder):
    for item in list(folder["folders"]):
        if "deleted" in folder["folders"][item]['action']:
            file = bf.get(folder["folders"][item]['path'])
            file.delete()
            del folder["folders"][item]
        else:
            recursive_folder_delete(folder["folders"][item])

    return

# Rename any folders that still exist.
def recursive_folder_rename(folder):
    for item in list(folder["folders"]):
        if "renamed" in folder["folders"][item]['action'] and folder["folders"][item]["type"] == "bf":
            file = bf.get(folder["folders"][item]["path"])
            file.name = item
            file.update()
    else:
        recursive_file_rename(folder["folders"][item])

    return

# 1. Remove all existing files on blackfynn, that the user deleted.
try: 
    main_curate_progress_message = "Deleting files on blackfynn"
    dataset_structure = soda_json_structure["dataset-structure"]
    recursive_file_delete(dataset_structure)
    main_curate_progress_message = "Files on blackfynn marked for deletion have been deleted"
except Exception as e:
    raise e

# 2. Get the status of all files currently on blackfynn and create 
# the folderpath for all items in both dataset structures.
try:
    main_curate_progress_message = "Retreiving files and folders from blackfynn"
    current_bf_dataset_files_folders = bf_import_dataset_to_json(soda_json_structure)[0]
    bfsd = current_bf_dataset_files_folders["dataset-structure"]
    main_curate_progress_message = "Creating file paths for all files on blackfynn"
    recursive_item_path_create(dataset_structure, [])
    recursive_item_path_create(bfsd, [])
    main_curate_progress_message = "File paths created"
except Exception as e:
    raise e

# 3. Move any files that are marked as moved on blackfynn. 
# Create any additional folders if required
try: 
    main_curate_progress_message = "Moving all files requested by the user"
    recursive_check_moved_files(dataset_structure)
    main_curate_progress_message = "Moved all files requested by the user"
except Exception as e:
    raise e

# 4. Rename any blackfynn files that are marked as renamed. 
try:
    main_curate_progress_message = "Renaming all files requested by the user"
    recursive_file_rename(dataset_structure)
    main_curate_progress_message = "Renamed all files requested by the user"
except Exception as e:
    raise e

# 5. Delete any blackfynn folders that are marked as deleted. 
try:
    main_curate_progress_message = "Deleting any additional folders present on blackfynn"
    recursive_folder_delete(dataset_structure)
    main_curate_progress_message = "Deletion of additional folders complete"
except Exception as e:
    raise e




<Collection name='primary' id='N:collection:95155ae6-df69-45f4-8493-0b988720c92a'>
<DataPackage name='manifest' id='N:package:f0573ab9-e78d-407c-b6d4-085f40cf5d7f'>
<DataPackage name='submission' id='N:package:3b504e70-58ca-4bfb-8c80-f214f32ccaa0'>
<DataPackage name='subjects' id='N:package:41295a11-4fe0-4d4d-95fb-26b50646f74b'>
<DataPackage name='dataset_description' id='N:package:69a01be0-42c3-4129-9898-4aa47734b4ea'>
<Collection name='source' id='N:collection:2994d497-37c0-41c9-b88a-1daff9a7c0bc'>
<Collection name='png' id='N:collection:252325e6-ee96-4a79-9379-1b3536d33877'>
<DataPackage name='soda_icon (1)' id='N:package:ed1ae9a2-5067-428a-9dde-65a76b5f2732'>
<DataPackage name='soda_icon2' id='N:package:ab3029d9-31f0-49d5-9e37-97cd1066692a'>
<DataPackage name='icon.ico' id='N:package:1d277d08-b129-4684-b611-1d33ddcdda2a'>
<Collection name='code' id='N:collection:cdeabbcd-47b6-4f0b-b500-56f5e67fc0b6'>
<Collection name='fhj' id='N:collection:67f32258-459e-4db6-a884-859ab38b2efb'>
<Da

In [146]:
folder = mysd
list(folder["folders"])


['primary', 'source', 'code']

In [ ]:
recursive_file_delete(mysd)
recursive_item_path_create(mysd, [])
recursive_item_path_create(bfsd, [])
recursive_check_moved_files(mysd)
recursive_file_rename(mysd)
recursive_folder_delete(mysd)